## Data Cleaning and Vectorization

In this notebook, we will do the following
1. Null Values Imputation : For every field, the null values will be replaced with median value. This is very simple and crude form of imputation, however the model based imputation is complicated to design, and hence will be implemented in the next iteration
2. Vectorization : After the imputation, data for each file will be processed as below<br>
For numerical columns, the values will be scaled between 0 and 1<br>
For categorical columns, the values will be encoded as one hot vectors

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# project directory
current_dir = 'Home Credit_Kaggle'

# set the project folder as current working directory
import os
complete_path = os.path.join('/content/drive/My Drive/Colab Notebooks/',current_dir)
os.chdir(complete_path)

In [ ]:
import numpy as np
import pandas as pd
import time
from scipy.sparse import csr_matrix, save_npz

#Load Control Files

##Load field level flags

Since there are a lot of files and fields, we have soft coded below conditional information regarding fields in files for easy maintenance. In case either of these conditions need to be changed, we only need to change the file and rerun the notebook to regenerate the data as per new conditions. No code change required!!

1. whether the field is to be used or not
2. is it a categorical or numerical or key field
3. is it to be normalized or not

In [ ]:
# load HomeCredit_Control File_Field level.csv
field_level_flags = pd.read_csv('control/HomeCredit_Control File_Field level.csv')
print(field_level_flags.shape)
field_level_flags.head()

(220, 5)


,FILE_NAME,FIELD_NAME,FIELD_TYPE,USE_FIELD,NORMALIZE_FIELD
0,application_train.csv,SK_ID_CURR,Primary Key,Y,N
1,application_train.csv,TARGET,Target Value,Y,N
2,application_train.csv,NAME_CONTRACT_TYPE,Categorical,Y,N
3,application_train.csv,CODE_GENDER,Categorical,Y,N
4,application_train.csv,FLAG_OWN_CAR,Categorical,Y,N


In [ ]:
# create a dictionary from above data using [FILE_NAME,FIELD_NAME] as key
# for fast lookup

# prepare key as 'FILE_NAME'+'FIELD_NAME' for each record
file_name_arr = np.asarray(field_level_flags['FILE_NAME'])
field_name_arr = np.asarray(field_level_flags['FIELD_NAME'])
l = len(file_name_arr)
keys = [(str(file_name_arr[i])+str(field_name_arr[i])).strip() for i in range(l)]

# prepare values as ['FIELD_TYPE','USE_FIELD','NORMALIZE_FIELD'] for each record
field_type_arr = np.asarray(field_level_flags['FIELD_TYPE'])
use_field_arr = np.asarray(field_level_flags['USE_FIELD'])
norm_field_arr = np.asarray(field_level_flags['NORMALIZE_FIELD'])
values = [[field_type_arr[i],use_field_arr[i],norm_field_arr[i]] for i in range(l)]

# combined into dictionary
dict_field_flags = dict(zip(keys,values))
print(dict_field_flags.keys())
print(dict_field_flags.values())

dict_keys(['application_train.csvSK_ID_CURR', 'application_train.csvTARGET', 'application_train.csvNAME_CONTRACT_TYPE', 'application_train.csvCODE_GENDER', 'application_train.csvFLAG_OWN_CAR', 'application_train.csvFLAG_OWN_REALTY', 'application_train.csvCNT_CHILDREN', 'application_train.csvAMT_INCOME_TOTAL', 'application_train.csvAMT_CREDIT', 'application_train.csvAMT_ANNUITY', 'application_train.csvAMT_GOODS_PRICE', 'application_train.csvNAME_TYPE_SUITE', 'application_train.csvNAME_INCOME_TYPE', 'application_train.csvNAME_EDUCATION_TYPE', 'application_train.csvNAME_FAMILY_STATUS', 'application_train.csvNAME_HOUSING_TYPE', 'application_train.csvREGION_POPULATION_RELATIVE', 'application_train.csvDAYS_BIRTH', 'application_train.csvDAYS_EMPLOYED', 'application_train.csvDAYS_REGISTRATION', 'application_train.csvDAYS_ID_PUBLISH', 'application_train.csvOWN_CAR_AGE', 'application_train.csvFLAG_MOBIL', 'application_train.csvFLAG_EMP_PHONE', 'application_train.csvFLAG_WORK_PHONE', 'application

## Load File Level Flags

The ORDER_BY flags loaded below will control the ordering of record in each file. Since the linking of files to each other is through keys, order of records is of utmost importance. It will help us to create file snapshots easily later.
The NUM_TOP_REC flags are used to control the number of records in File Snapshots. This flag will not be used in this notebook, it will be used later.

In [ ]:
# load HomeCredit_Control File_File Level.csv
file_level_flags = pd.read_csv('control/HomeCredit_Control File_File Level_nn.csv')
print(file_level_flags.shape)
file_level_flags.head(6)

(6, 4)


,FILE_NAME,NUM_TOP_REC,ORDER_BY,ASC ORDER?
0,bureau.csv,2,"SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT",1
1,bureau_balance.csv,1,"SK_ID_BUREAU,MONTHS_BALANCE",1
2,previous_application.csv,1,"SK_ID_CURR,SK_ID_PREV,DAYS_DECISION",1
3,POS_CASH_balance.csv,10,"SK_ID_CURR,SK_ID_PREV,MONTHS_BALANCE",1
4,installments_payments.csv,5,"SK_ID_CURR,SK_ID_PREV,DAYS_INSTALMENT",1
5,credit_card_balance.csv,10,"SK_ID_CURR,SK_ID_PREV,MONTHS_BALANCE",1


In [ ]:
# create a dictionary from above data using [FILE_NAME,FIELD_NAME] as key
# for fast lookup

# prepare key as 'FILE_NAME' for each record
file_name_arr = np.asarray(file_level_flags['FILE_NAME'])
l = len(file_name_arr)
keys = [str(file_name_arr[i]).strip() for i in range(l)]

# prepare values as ['NUM_TOP_REC','ORDER_BY','ASC_ORDER?'] for each record
num_top_rec_arr = np.asarray(file_level_flags['NUM_TOP_REC'])
order_by_arr = np.asarray(file_level_flags['ORDER_BY'])
asc_order_arr = np.asarray(file_level_flags['ASC ORDER?'])
values = [[num_top_rec_arr[i],order_by_arr[i],asc_order_arr[i]] for i in range(l)]

# combined into dictionary
dict_file_flags = dict(zip(keys,values))
print(dict_file_flags.keys())
print(dict_file_flags.values())

dict_keys(['bureau.csv', 'bureau_balance.csv', 'previous_application.csv', 'POS_CASH_balance.csv', 'installments_payments.csv', 'credit_card_balance.csv'])
dict_values([[2, 'SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT', 1], [1, 'SK_ID_BUREAU,MONTHS_BALANCE', 1], [1, 'SK_ID_CURR,SK_ID_PREV,DAYS_DECISION', 1], [10, 'SK_ID_CURR,SK_ID_PREV,MONTHS_BALANCE', 1], [5, 'SK_ID_CURR,SK_ID_PREV,DAYS_INSTALMENT', 1], [10, 'SK_ID_CURR,SK_ID_PREV,MONTHS_BALANCE', 1]])


##Create functions to preprocess data in files

We have defined three functions below to clean + vectorize/normalize the data in all files.

1. preprocess_categ_train => This function will impute + vectorize a categorical column of train data
2. preprocess_numeric_train => This function will impute + normalize (scale between 0 to 1) a numerical column of data
3. preprocess_file => This function will call above two functions for each file

In [ ]:
# function to impute and preprocess categorical data
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

def preprocess_categ_train(arr_train):

  # reshape array to be 2D
  arr_train_2D = np.asarray(arr_train).reshape(-1,1)

  # Part 1 - Impute with most frequent value
  # initialize imputer
  imputer = SimpleImputer(strategy='most_frequent')  
  imputer.fit(arr_train_2D)
  arr_train2 = imputer.transform(arr_train_2D)

  # reshape array to be 1D for vectorizer
  #arr_train2 = np.asarray(arr_train2).reshape(-1,)
  #print(arr_train2)

  # Part 2 - Encode the categorical values
  # initialize vectorizer
  count_vect = OneHotEncoder(handle_unknown='ignore')

  # fit vectorizer to training data for each categorical column
  # and use it to transform the training data
  count_vect.fit(arr_train2)
  train_values = count_vect.categories_[0] # store list of unique values
  #print(train_values)

  feat_size = len(count_vect.categories_[0]) # find no of unique values
  arr_train_ohe = count_vect.transform(arr_train2).toarray()

  return imputer,count_vect,feat_size,arr_train_ohe
##=========================end of function========================##

In [ ]:
# function to impute and preprocess numerical data
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

def preprocess_numeric_train(arr_train):

  # reshape array to be 2D
  arr_train_2D = np.asarray(arr_train).reshape(-1,1)

  # Part 1 - Impute with median value
  # initialize imputer
  imputer = SimpleImputer(strategy='median')
  # fit and transform with imputer
  imputer.fit(arr_train_2D)
  arr_train2 = imputer.transform(arr_train_2D)

  # Part 2 - Min Max scaling
  # initializer scaler
  field_scaler = MinMaxScaler(feature_range=(1e-3, 1))
  # fit and transform with scaler
  field_scaler.fit(arr_train2)
  arr_train_scaled = field_scaler.transform(arr_train2)

  #return scaler and scaled data
  return imputer,field_scaler,arr_train_scaled

In [ ]:
# function to preprocess a file

def preprocess_file(file_name,file_df,dict_field_flags):
  # preprocess file and return 3 chunks of data
  # key values => key_fields (dataframe)
  key_values = pd.DataFrame()
  # numerical data => numeric_data (numpy 2D array)
  numeric_data = np.array([[]])
  # categorial data => categ_data (numpy 2D array)
  categ_data = np.array([[]])

  # dict_preprocessors is the
  # dictionary to hold preprocessors for each field
  # one hot encoders for categorical data
  # scalers for numerical data
  dict_preprocessors = {}

  # same is dict_imputers for imputers
  dict_imputers = {}

  # dict of column processing order (column index) for each numerical field
  dict_col_index = {}
  col_index = 1 # init

  # dict of feature sizes for each categorical field
  dict_feat_size = {}

  # for each column of the df
  for col in file_df.columns:
    # look up the value of flags in dictionary
    field_key = str(file_name) + str(col)
    field_type, use_field, normalize_field = dict_field_flags[field_key]
    
    #print(file_df[col].shape)
    # if field is to be used
    if use_field != 'N':

      # if field is numerical
      if field_type == 'Numerical':
        # impute and preprocess field
        field_imputer,field_scaler,field_scaled = preprocess_numeric_train(file_df[col])
        #print(field_scaled.shape) 
        # append the scaler, column index and imputer to dictionary
        dict_preprocessors.update({field_key:field_scaler})
        dict_col_index.update({field_key:col_index})
        col_index += 1        
        dict_imputers.update({field_key:field_imputer})        

        # append the preprocessed numeric data to array
        if numeric_data.shape == (1,0): #first array being appended
          #print(numeric_data.shape)
          numeric_data = field_scaled
        else: 
          numeric_data = np.append(numeric_data,field_scaled,axis=1)
          #print(numeric_data.shape)

      # if field is categorical
      elif field_type == 'Categorical':
        # preprocess field
        field_imputer,field_vect,feat_size,field_ohe = preprocess_categ_train(file_df[col])
        #print(field_ohe.shape) 

        # append the vectorizer, feature size and imputer to dictionary
        dict_preprocessors.update({field_key:field_vect})
        dict_feat_size.update({field_key:feat_size})
        dict_imputers.update({field_key:field_imputer})                

        # append the preprocessed categorical data to array
        if categ_data.shape == (1,0): #first array being appended
          categ_data = field_ohe
        else:
          categ_data = np.append(categ_data,field_ohe,axis=1)
        

      # if field is a key or target value
      elif field_type == 'Primary Key' or field_type == 'Foreign Key' or field_type == 'Target Value':
        # append key column to dataframe
        key_values[col] = file_df[col]
      #==========end of if elif block============#
    #===========end of use_field=='Y' block==========#
  #=======================end of for loop================#

  return key_values,numeric_data,categ_data,dict_preprocessors,dict_feat_size,dict_col_index,dict_imputers

##Create folders for preprocessed outputs and preprocessors

The preprocessed output (scaled numerical fields and one hot encoded categorical fields) will be stored in <I>preprocesssed</I> folder. The scalers and encoders for the same will be stored in <I>preprocessors</I> folder. Create these folders if not already present.

In [ ]:
# create output folder for preprocessed data if not already present
out_path_data = os.path.join(complete_path,'preprocessed')
if not os.path.isdir(out_path_data):
  os.mkdir(out_path_data)

# create output folder for preprocessors if not already present
out_path_preprocessors = os.path.join(complete_path,'preprocessors')
if not os.path.isdir(out_path_preprocessors):
  os.mkdir(out_path_preprocessors)

## Call above functions for each file

###Application Train File

In [ ]:
# size check
file_df = pd.read_csv('data/application_train.csv')
file_df.shape

(307511, 122)

In [ ]:
# start time
s_time = time.time()
# init file name
file_name = 'application_train.csv'
# load file into df
file_df = pd.read_csv('data/application_train.csv')
app_train_keys,app_train_numeric_data,app_train_categ_data,app_train_preprocessors,app_train_feat_size,app_train_col_index,app_train_imputers = preprocess_file(file_name,file_df,dict_field_flags)
print("Time Taken (in seconds): ",(time.time() - s_time))

Time Taken (in seconds):  943.5748243331909


In [ ]:
print(app_train_keys.head())
print(app_train_keys.shape)
print(app_train_numeric_data.shape)
print(app_train_categ_data.shape)
print(app_train_feat_size)
print(app_train_col_index)

   SK_ID_CURR  TARGET
0      100002       1
1      100003       0
2      100004       0
3      100006       0
4      100007       0
(307511, 2)
(307511, 27)
(307511, 188)
{'application_train.csvNAME_CONTRACT_TYPE': 2, 'application_train.csvCODE_GENDER': 3, 'application_train.csvFLAG_OWN_CAR': 2, 'application_train.csvFLAG_OWN_REALTY': 2, 'application_train.csvNAME_TYPE_SUITE': 7, 'application_train.csvNAME_INCOME_TYPE': 8, 'application_train.csvNAME_EDUCATION_TYPE': 5, 'application_train.csvNAME_FAMILY_STATUS': 6, 'application_train.csvNAME_HOUSING_TYPE': 6, 'application_train.csvFLAG_MOBIL': 2, 'application_train.csvFLAG_EMP_PHONE': 2, 'application_train.csvFLAG_WORK_PHONE': 2, 'application_train.csvFLAG_CONT_MOBILE': 2, 'application_train.csvFLAG_PHONE': 2, 'application_train.csvFLAG_EMAIL': 2, 'application_train.csvOCCUPATION_TYPE': 18, 'application_train.csvWEEKDAY_APPR_PROCESS_START': 7, 'application_train.csvREG_REGION_NOT_LIVE_REGION': 2, 'application_train.csvREG_REGION_NOT_WOR

####Checking categorical columns to make sure the values have been encoded correctly

An easy way to check that one hot encoding is working correctly, is verifying the no of distinct values in the final feature size, against the output of EDA_Basic notebook for the file. Below output correctly matches with output for application_train.csv.

In [ ]:
# quick check of categorical values
s = 0
print("\t\t File NameFieldName \t\t    No of    Cumulative sum")
print("                   \t\t\t\t dist. values              ")
for i,v in app_train_feat_size.items():
  s += v
  print("{:50} \t {:2} \t {:2}".format(i,v,s))

		 File NameFieldName 		    No of    Cumulative sum
                   				 dist. values              
application_train.csvNAME_CONTRACT_TYPE            	  2 	  2
application_train.csvCODE_GENDER                   	  3 	  5
application_train.csvFLAG_OWN_CAR                  	  2 	  7
application_train.csvFLAG_OWN_REALTY               	  2 	  9
application_train.csvNAME_TYPE_SUITE               	  7 	 16
application_train.csvNAME_INCOME_TYPE              	  8 	 24
application_train.csvNAME_EDUCATION_TYPE           	  5 	 29
application_train.csvNAME_FAMILY_STATUS            	  6 	 35
application_train.csvNAME_HOUSING_TYPE             	  6 	 41
application_train.csvFLAG_MOBIL                    	  2 	 43
application_train.csvFLAG_EMP_PHONE                	  2 	 45
application_train.csvFLAG_WORK_PHONE               	  2 	 47
application_train.csvFLAG_CONT_MOBILE              	  2 	 49
application_train.csvFLAG_PHONE                    	  2 	 51
application_train.csvFLAG_EMAIL            

In [ ]:
# save the above outputs to drive

app_train_keys.to_csv('preprocessed/app_train_keys.csv',index=False)
np.save("preprocessed/app_train_numeric_data",app_train_numeric_data)
#np.save("preprocessed/app_train_categ_data",app_train_categ_data)
app_train_categ_data_csr = csr_matrix(app_train_categ_data)
save_npz('preprocessed/app_train_categ_data_csr.npz',app_train_categ_data_csr)

import pickle
app_train_preprocessors_file = open('preprocessors/app_train_preprocessors','wb')
pickle.dump(app_train_preprocessors,app_train_preprocessors_file)
app_train_preprocessors_file.close()

app_train_feat_size_file = open('preprocessors/app_train_feat_size','wb')
pickle.dump(app_train_feat_size,app_train_feat_size_file)
app_train_feat_size_file.close()

app_train_imputers_file = open('preprocessors/app_train_imputers','wb')
pickle.dump(app_train_imputers,app_train_imputers_file)
app_train_imputers_file.close()

app_train_col_index_file = open('preprocessors/app_train_col_index','wb')
pickle.dump(app_train_col_index,app_train_col_index_file)
app_train_col_index_file.close()

###Previous Application.csv

In [ ]:
# size check
file_df = pd.read_csv('data/previous_application.csv')
file_df.shape

(1670214, 37)

In [ ]:
# start time
s_time = time.time()
# init file name
file_name = 'previous_application.csv'
# load file into df
#file_df = pd.read_csv('data/previous_application.csv',nrows=1000)
file_df = pd.read_csv('data/previous_application.csv')
#print(file_df.head(10))

# order the file by key fields and the ordering key
sort_keys = dict_file_flags[file_name][1].split(',') # split the string into list of key fields
asc_order = list(dict_file_flags[file_name][2]**range(len(sort_keys))) # flags to control if dataframe should be sorted in asc order
# list was required above since one flag is required for each key

file_df.sort_values(by=sort_keys,ascending=asc_order,inplace=True,na_position='last')
file_df.reset_index(drop=True,inplace=True)
#print(file_df.head(10))

prev_app_keys,prev_app_numeric_data,prev_app_categ_data,prev_app_preprocessors,prev_app_feat_size,
prev_app_imputers = preprocess_file(file_name,file_df,dict_field_flags)
print("Time Taken (in seconds): ",(time.time() - s_time))

In [ ]:
print(prev_app_keys.head())
print(prev_app_keys.shape)
print(prev_app_numeric_data.shape)
print(prev_app_categ_data.shape)
print(prev_app_feat_size)
print(prev_app_col_index)

In [ ]:
# save the above outputs to drive
prev_app_keys.to_csv('preprocessed/prev_app_keys.csv',index=False)
np.save("preprocessed/prev_app_numeric_data",prev_app_numeric_data)
#np.save("preprocessed/prev_app_categ_data",prev_app_categ_data)
prev_app_categ_data_csr = csr_matrix(prev_app_categ_data)
save_npz('preprocessed/prev_app_categ_data_csr.npz',prev_app_categ_data_csr)

import pickle
prev_app_preprocessors_file = open('preprocessors/prev_app_preprocessors','wb')
pickle.dump(prev_app_preprocessors,prev_app_preprocessors_file)
prev_app_preprocessors_file.close()

prev_app_feat_size_file = open('preprocessors/prev_app_feat_size','wb')
pickle.dump(prev_app_feat_size,prev_app_feat_size_file)
prev_app_feat_size_file.close()

prev_app_col_index_file = open('preprocessors/prev_app_col_index','wb')
pickle.dump(prev_app_col_index,prev_app_col_index_file)
prev_app_col_index_file.close()

###Bureau.csv

In [ ]:
# size check
file_df = pd.read_csv('data/bureau.csv')
file_df.shape

(1716428, 17)

In [ ]:
# start time
s_time = time.time()
# init file name
file_name = 'bureau.csv'
# load file into df
#file_df = pd.read_csv('data/bureau.csv',nrows=1000)
file_df = pd.read_csv('data/bureau.csv')
#print(file_df.head(10))

# order the file by key fields and the ordering key

# get the keys and sorting order
sort_keys = dict_file_flags[file_name][1].split(',') # split the string into list of key fields
asc_order = list(dict_file_flags[file_name][2]**range(len(sort_keys))) # flags to control if dataframe should be sorted in asc order
# list was required above since one flag is required for each key

# do the sorting
file_df.sort_values(by=sort_keys,ascending=asc_order,inplace=True,na_position='last')
file_df.reset_index(drop=True,inplace=True)
#print(file_df.head(10))

bureau_keys,bureau_numeric_data,bureau_categ_data,bureau_preprocessors,bureau_feat_size,bureau_col_index,bureau_imputers = preprocess_file(file_name,file_df,dict_field_flags)
print("Time Taken (in seconds): ",(time.time() - s_time))

Time Taken (in seconds):  10052.490975856781


In [ ]:
print(bureau_keys.head())
print(bureau_keys.shape)
print(bureau_numeric_data.shape)
print(bureau_categ_data.shape)
print(bureau_feat_size)
print(bureau_col_index)

   SK_ID_CURR  SK_ID_BUREAU
0      100001       5896630
1      100001       5896631
2      100001       5896632
3      100001       5896633
4      100001       5896634
(1716428, 2)
(1716428, 10)
(1716428, 23)
{'bureau.csvCREDIT_ACTIVE': 4, 'bureau.csvCREDIT_CURRENCY': 4, 'bureau.csvCREDIT_TYPE': 15}


In [ ]:
# save the above outputs to drive
bureau_keys.to_csv('preprocessed/bureau_keys.csv',index=False)
np.save("preprocessed/bureau_numeric_data",bureau_numeric_data)
#np.save("preprocessed/bureau_categ_data",bureau_categ_data)
bureau_categ_data_csr = csr_matrix(bureau_categ_data)
save_npz('preprocessed/bureau_categ_data_csr.npz',bureau_categ_data_csr)

import pickle
bureau_preprocessors_file = open('preprocessors/bureau_preprocessors','wb')
pickle.dump(bureau_preprocessors,bureau_preprocessors_file)
bureau_preprocessors_file.close()

bureau_feat_size_file = open('preprocessors/bureau_feat_size','wb')
pickle.dump(bureau_feat_size,bureau_feat_size_file)
bureau_feat_size_file.close()

bureau_col_index_file = open('preprocessors/bureau_col_index','wb')
pickle.dump(bureau_col_index,bureau_col_index_file)
bureau_col_index_file.close()

###Bureau Balance.csv

In [ ]:
# size check
file_df = pd.read_csv('data/bureau_balance.csv')
file_df.shape

(27299925, 3)

In [ ]:
# start time
s_time = time.time()
# init file name
file_name = 'bureau_balance.csv'
# load file into df
#file_df = pd.read_csv('data/bureau_balance.csv',nrows=1000)
file_df = pd.read_csv('data/bureau_balance.csv')

# take only a part of data as there are 27M records!!   
num_of_rows_to_keep = len(file_df)//2
file_df = file_df[:num_of_rows_to_keep]
#print(file_df.head(10))

# order the file by key fields and the ordering key

# get the keys and sorting order
sort_keys = dict_file_flags[file_name][1].split(',') # split the string into list of key fields
asc_order = list(dict_file_flags[file_name][2]**range(len(sort_keys))) # flags to control if dataframe should be sorted in asc order
# list was required above since one flag is required for each key

# do the sorting
file_df.sort_values(by=sort_keys,ascending=asc_order,inplace=True,na_position='last')
file_df.reset_index(drop=True,inplace=True)
#print(file_df.head(10))

bureau_bal_keys,bureau_bal_numeric_data,bureau_bal_categ_data,bureau_bal_preprocessors,bureau_bal_feat_size,bureau_bal_col_index,bureau_bal_imputers = preprocess_file(file_name,file_df,dict_field_flags)
print("Time Taken (in seconds): ",(time.time() - s_time))

Time Taken (in seconds):  22988.762111902237


In [ ]:
print(bureau_bal_keys.head())
print(bureau_bal_keys.shape)
print(bureau_bal_numeric_data.shape)
print(bureau_bal_categ_data.shape)
print(bureau_bal_feat_size)
print(bureau_bal_col_index)

   SK_ID_BUREAU
0       5001710
1       5001710
2       5001710
3       5001710
4       5001710
(13649962, 1)
(13649962, 1)
(13649962, 8)
{'bureau_balance.csvSTATUS': 8}


In [ ]:
# save the above outputs to drive
bureau_bal_keys.to_csv('preprocessed/bureau_bal_keys.csv',index=False)
np.save("preprocessed/bureau_bal_numeric_data",bureau_bal_numeric_data)
#np.save("preprocessed/bureau_bal_categ_data",bureau_bal_categ_data)
bureau_bal_categ_data_csr = csr_matrix(bureau_bal_categ_data)
save_npz('preprocessed/bureau_bal_categ_data_csr.npz',bureau_bal_categ_data_csr)

import pickle
bureau_bal_preprocessors_file = open('preprocessors/bureau_bal_preprocessors','wb')
pickle.dump(bureau_bal_preprocessors,bureau_bal_preprocessors_file)
bureau_bal_preprocessors_file.close()

bureau_bal_feat_size_file = open('preprocessors/bureau_bal_feat_size','wb')
pickle.dump(bureau_bal_feat_size,bureau_bal_feat_size_file)
bureau_bal_feat_size_file.close()

bureau_bal_col_index_file = open('preprocessors/bureau_bal_col_index','wb')
pickle.dump(bureau_bal_col_index,bureau_bal_col_index_file)
bureau_bal_col_index_file.close()

###POS Cash Balance.csv

In [ ]:
# size check
file_df = pd.read_csv('data/POS_CASH_balance.csv')
file_df.shape

(10001358, 8)

In [ ]:
# start time
s_time = time.time()
# init file name
file_name = 'POS_CASH_balance.csv'
# load file into df
#file_df = pd.read_csv('data/POS_CASH_balance.csv',nrows=1000)
file_df = pd.read_csv('data/POS_CASH_balance.csv')
#print(file_df.head(10))

# order the file by key fields and the ordering key

# get the keys and sorting order
sort_keys = dict_file_flags[file_name][1].split(',') # split the string into list of key fields
asc_order = list(dict_file_flags[file_name][2]**range(len(sort_keys))) # flags to control if dataframe should be sorted in asc order
# use only a part of the dataset, since original file has 10 Million records!!
num_of_rows_to_keep = len(file_df)//2
file_df = file_df[:num_of_rows_to_keep]
# list was required above since one flag is required for each key

# do the sorting
file_df.sort_values(by=sort_keys,ascending=asc_order,inplace=True,na_position='last')
file_df.reset_index(drop=True,inplace=True)
#print(file_df.head(10))

pos_cash_bal_keys,pos_cash_bal_numeric_data,pos_cash_bal_categ_data,pos_cash_bal_preprocessors,pos_cash_bal_feat_size,pos_cash_bal_col_index,pos_cash_bal_imputers = preprocess_file(file_name,file_df,dict_field_flags)
print("Time Taken (in seconds): ",(time.time() - s_time))

Time Taken (in seconds):  19628.401156663895


In [ ]:
print(pos_cash_bal_keys.head())
print(pos_cash_bal_keys.shape)
print(pos_cash_bal_numeric_data.shape)
print(pos_cash_bal_categ_data.shape)
print(pos_cash_bal_feat_size)
print(pos_cash_bal_col_index)

   SK_ID_PREV  SK_ID_CURR
0     1369693      100001
1     1369693      100001
2     1851984      100001
3     1851984      100001
4     1851984      100001
(5000679, 2)
(5000679, 5)
(5000679, 8)
{'POS_CASH_balance.csvNAME_CONTRACT_STATUS': 8}


In [ ]:
# save the above outputs to drive
pos_cash_bal_keys.to_csv('preprocessed/pos_cash_bal_keys.csv',index=False)
np.save("preprocessed/pos_cash_bal_numeric_data",pos_cash_bal_numeric_data)
#np.save("preprocessed/pos_cash_bal_categ_data",pos_cash_bal_categ_data)
pos_cash_bal_categ_data_csr = csr_matrix(pos_cash_bal_categ_data)
save_npz('preprocessed/pos_cash_bal_categ_data_csr.npz',pos_cash_bal_categ_data_csr)

import pickle
pos_cash_bal_preprocessors_file = open('preprocessors/pos_cash_bal_preprocessors','wb')
pickle.dump(pos_cash_bal_preprocessors,pos_cash_bal_preprocessors_file)
pos_cash_bal_preprocessors_file.close()

pos_cash_bal_feat_size_file = open('preprocessors/pos_cash_bal_feat_size','wb')
pickle.dump(pos_cash_bal_feat_size,pos_cash_bal_feat_size_file)
pos_cash_bal_feat_size_file.close()

pos_cash_bal_col_index_file = open('preprocessors/pos_cash_bal_col_index','wb')
pickle.dump(pos_cash_bal_col_index,pos_cash_bal_col_index_file)
pos_cash_bal_col_index_file.close()

###Installments Payments.csv

In [ ]:
# size check
file_df = pd.read_csv('data/installments_payments.csv')
file_df.shape

(13605401, 8)

In [ ]:
# start time
s_time = time.time()
# init file name
file_name = 'installments_payments.csv'
# load file into df
#file_df = pd.read_csv('data/installments_payments.csv',nrows=1000)
file_df = pd.read_csv('data/installments_payments.csv')
#print(file_df.head(10))

# order the file by key fields and the ordering key

# get the keys and sorting order
sort_keys = dict_file_flags[file_name][1].split(',') # split the string into list of key fields
asc_order = list(dict_file_flags[file_name][2]**range(len(sort_keys))) # flags to control if dataframe should be sorted in asc order
# list was required above since one flag is required for each key

# do the sorting
file_df.sort_values(by=sort_keys,ascending=asc_order,inplace=True,na_position='last')
file_df.reset_index(drop=True,inplace=True)
#print(file_df.head(10))

instalm_paym_keys,instalm_paym_numeric_data,instalm_paym_categ_data,instalm_paym_preprocessors,instalm_paym_feat_size,instalm_paym_col_index,instalm_paym_imputers = preprocess_file(file_name,file_df,dict_field_flags)
print("Time Taken (in seconds): ",(time.time() - s_time))

Time Taken (in seconds):  37.459702014923096


In [ ]:
print(instalm_paym_keys.head())
print(instalm_paym_keys.shape)
print(instalm_paym_numeric_data.shape)
print(instalm_paym_categ_data.shape)
print(instalm_paym_feat_size)
print(instalm_paym_col_index)

   SK_ID_PREV  SK_ID_CURR
0     1369693      100001
1     1369693      100001
2     1369693      100001
3     1369693      100001
4     1851984      100001
(13605401, 2)
(13605401, 6)
(1, 0)
{}


In [ ]:
# save the above outputs to drive
instalm_paym_keys.to_csv('preprocessed/instalm_paym_keys.csv',index=False)
np.save("preprocessed/instalm_paym_numeric_data",instalm_paym_numeric_data)
#np.save("preprocessed/instalm_paym_categ_data",instalm_paym_categ_data) # no categ data for this file

import pickle
instalm_paym_preprocessors_file = open('preprocessors/instalm_paym_preprocessors','wb')
pickle.dump(instalm_paym_preprocessors,instalm_paym_preprocessors_file)
instalm_paym_preprocessors_file.close()

instalm_paym_feat_size_file = open('preprocessors/instalm_paym_feat_size','wb')
pickle.dump(instalm_paym_feat_size,instalm_paym_feat_size_file)
instalm_paym_feat_size_file.close()

instalm_paym_col_index_file = open('preprocessors/instalm_paym_col_index','wb')
pickle.dump(instalm_paym_col_index,instalm_paym_col_index_file)
instalm_paym_col_index_file.close()

###Credit Card Balance.csv

In [ ]:
# size check
file_df = pd.read_csv('data/credit_card_balance.csv')
file_df.shape

(3840312, 23)

In [ ]:
# start time
s_time = time.time()
# init file name
file_name = 'credit_card_balance.csv'
# load file into df
#file_df = pd.read_csv('data/credit_card_balance.csv',nrows=1000)
file_df = pd.read_csv('data/credit_card_balance.csv')
#print(file_df.head(10))

# order the file by key fields and the ordering key

# get the keys and sorting order
sort_keys = dict_file_flags[file_name][1].split(',') # split the string into list of key fields
asc_order = list(dict_file_flags[file_name][2]**range(len(sort_keys))) # flags to control if dataframe should be sorted in asc order
# list was required above since one flag is required for each key

# do the sorting
file_df.sort_values(by=sort_keys,ascending=asc_order,inplace=True,na_position='last')
file_df.reset_index(drop=True,inplace=True)
#print(file_df.head(10))

credit_bal_keys,credit_bal_numeric_data,credit_bal_categ_data,credit_bal_preprocessors,credit_bal_feat_size,credit_bal_col_index,credit_bal_imputers = preprocess_file(file_name,file_df,dict_field_flags)
print("Time Taken (in seconds): ",(time.time() - s_time))

Time Taken (in seconds):  12073.953548192978
Time Taken (in seconds):  12073.953548192978


In [ ]:
print(credit_bal_keys.head())
print(credit_bal_keys.shape)
print(credit_bal_numeric_data.shape)
print(credit_bal_categ_data.shape)
print(credit_bal_feat_size)
print(credit_bal_col_index)

   SK_ID_PREV  SK_ID_CURR
0     1489396      100006
1     1489396      100006
2     1489396      100006
3     1489396      100006
4     1489396      100006
(3840312, 2)
(3840312, 20)
(3840312, 7)
{'credit_card_balance.csvNAME_CONTRACT_STATUS': 7}
   SK_ID_PREV  SK_ID_CURR
0     1489396      100006
1     1489396      100006
2     1489396      100006
3     1489396      100006
4     1489396      100006
(3840312, 2)
(3840312, 20)
(3840312, 7)
{'credit_card_balance.csvNAME_CONTRACT_STATUS': 7}


In [ ]:
# save the above outputs to drive
credit_bal_keys.to_csv('preprocessed/credit_bal_keys.csv',index=False)
np.save("preprocessed/credit_bal_numeric_data",credit_bal_numeric_data)
#np.save("preprocessed/credit_bal_categ_data",credit_bal_categ_data)
credit_bal_categ_data_csr = csr_matrix(credit_bal_categ_data)
save_npz('preprocessed/credit_bal_categ_data_csr.npz',credit_bal_categ_data_csr)

import pickle
credit_bal_preprocessors_file = open('preprocessors/credit_bal_preprocessors','wb')
pickle.dump(credit_bal_preprocessors,credit_bal_preprocessors_file)
credit_bal_preprocessors_file.close()

credit_bal_feat_size_file = open('preprocessors/credit_bal_feat_size','wb')
pickle.dump(credit_bal_feat_size,credit_bal_feat_size_file)
credit_bal_feat_size_file.close()

credit_bal_col_index_file = open('preprocessors/credit_bal_col_index','wb')
pickle.dump(credit_bal_col_index,credit_bal_col_index_file)
credit_bal_col_index_file.close()